<a href="https://colab.research.google.com/github/mercadoerik1031/snn-sound-localization/blob/write_to_disk/snn_sound_localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**SNN Sounnd Localization**



---



# Pip Installs

In [18]:
! pip install snntorch --quiet

# Imports

In [19]:
import pandas as pd
import os
import librosa
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from snntorch import spikegen
from brian2 import *
from brian2hears import *
import gc
from concurrent.futures import ThreadPoolExecutor

In [20]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Config

In [21]:
config = {
    # Google Colab Path
    "metadata_path": "/content/drive/My Drive/Colab Notebooks/Masters Project/metadata.parquet",
    "ambisonics_path": "/content/drive/My Drive/Colab Notebooks/Masters Project/spatial_librispeech_sample/ambisonics_sample",
    "noise_path": "/content/drive/My Drive/Colab Notebooks/Masters Project/spatial_librispeech_sample/noise_ambisonics_sample",
    "output_path": "/content/drive/My Drive/Colab Notebooks/Masters Project/spatial_librispeech_sample/preprocessed_samples",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "batch_size_pre": 32,
    "sr": 16000,

    "time_based_encoding": True,
    "num_steps": 10,
    "max_rate": 10,
    "noise": True,
}

# Filter Data

In [22]:
def filter_data(metadata_path=config["metadata_path"], ambisonics_path=config["ambisonics_path"], noise_path=config["noise_path"]):
  # Load metadata
  metadata = pd.read_parquet(metadata_path, engine="pyarrow")

  # Get lists of all files in directories
  ambisonic_files = [f for f in os.listdir(ambisonics_path) if os.path.isfile(os.path.join(ambisonics_path, f))]
  noise_files = [f for f in os.listdir(noise_path) if os.path.isfile(os.path.join(noise_path, f))]

  # Extract sample ids from filenames and filter metadata
  sample_ids = [int(f.split(".")[0].lstrip("0") or 0) for f in ambisonic_files]
  filtered_metadata = metadata[metadata["sample_id"].isin(sample_ids)]

  # Create full file paths
  ambisonic_files = [os.path.join(ambisonics_path, f) for f in ambisonic_files]
  noise_files = [os.path.join(noise_path, f) for f in noise_files]

  return filtered_metadata, ambisonic_files, noise_files


# Preprocess Functions

## Normalize

In [24]:
def normalize(audio_data, device=config["device"]):
  audio_data = audio_data.to(device)
  return (audio_data - audio_data.min()) / (audio_data.max() - audio_data.min())


## Rate Based Encoding

In [25]:
def rate_based_encoding(audio_data, max_rate=config["max_rate"], num_steps=config["num_steps"], device=config["device"]):
    if audio_data is None:
        raise ValueError("Input data is None.")

    # Check if audio_data is already a tensor, if not convert it
    if not isinstance(audio_data, torch.Tensor):
        audio_data = torch.tensor(audio_data, device=device)

    audio_data = audio_data.float().to(device)

    normalized_data = normalize(audio_data)

    spike_rates = normalized_data * max_rate

    spike_train = spikegen.rate(spike_rates, num_steps=num_steps)

    return spike_train



## Time Based Encoding

In [26]:
def time_based_encoding(audio_data, num_steps=config["num_steps"], device=config["device"]):
    if audio_data is None:
        raise ValueError("Input data is None.")

    # Check if audio_data is already a tensor, if not convert it
    if not isinstance(audio_data, torch.Tensor):
        audio_data = torch.tensor(audio_data, device=device)

    audio_data = audio_data.float().to(device)

    normalized_data = normalize(audio_data)

    spike_times = torch.where(normalized_data > 0.5, 1, 0)

    spike_train = spikegen.latency(spike_times, num_steps=num_steps, bypass=True)

    return spike_train


## Preprocess Function

In [27]:
def preprocess(ambisonic_file, noise_file, duration, device=config["device"], sr=config["sr"]):
    # Load ambisonic audio directly to GPU if possible
    audio = torch.tensor(librosa.load(ambisonic_file, sr=sr, mono=False, duration=duration)[0], device=device)
    length = int(np.round(duration * sr))

    # Pad Ambisonic File
    padded_ambisonic = torch.nn.functional.pad(audio, (0, max(0, length - audio.shape[1])))

    # Combine Noise (Optional)
    if config["noise"] and noise_file:
        # Load Noise File
        noise_audio = torch.tensor(librosa.load(noise_file, sr=sr, mono=False, duration=duration)[0], device=device)

        # Pad Noise File
        padded_noise = torch.nn.functional.pad(noise_audio, (0, max(0, length - noise_audio.shape[1])))

        # Combine Ambisonic & Noise
        combined_audio = padded_ambisonic + padded_noise
    else:
        combined_audio = padded_ambisonic

    # Processed_audio should be processed on GPU
    spike_trains = time_based_encoding(combined_audio) if config["time_based_encoding"] else rate_based_encoding(combined_audio)

    return spike_trains



## Process & Save Batches

In [30]:
def process_batch(batch_ambisonic_files, batch_noise_files, batch_metadata, output_path, duration, sr, batch_id):
    processed_data = []
    labels = []

    # Process each file in the batch
    for ambisonic_file, noise_file, meta_row in zip(batch_ambisonic_files, batch_noise_files, batch_metadata.itertuples()):
        spike_trains = preprocess(ambisonic_file, noise_file, duration)
        processed_data.append(spike_trains.cpu())

        labels.append({
            'sample_id': meta_row.sample_id,
            'split': meta_row.split,
            'azimuth': batch_metadata.at[meta_row.Index, 'speech/azimuth'],
            'elevation': batch_metadata.at[meta_row.Index, 'speech/elevation']
        })

    # Save processed data and labels
    batch_data_filename = f'processed_batch_{batch_id}.pt'
    batch_labels_filename = f'labels_batch_{batch_id}.csv'
    torch.save(torch.stack(processed_data), os.path.join(output_path, batch_data_filename))
    pd.DataFrame(labels).to_csv(os.path.join(output_path, batch_labels_filename), index=False)

    print(f"Batch {batch_id} processed and saved.")

def parallel_process_batches(metadata, ambisonic_files, noise_files, duration, batch_size=config["batch_size_pre"], output_path=config["output_path"], sr=config["sr"], max_workers=4):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for i in range(0, len(ambisonic_files), batch_size):
            batch_ambisonic_files = ambisonic_files[i:i+batch_size]
            batch_noise_files = noise_files[i:i+batch_size]
            batch_metadata = metadata.iloc[i:i+batch_size]

            # Submit batch processing jobs to the executor
            futures.append(executor.submit(process_batch, batch_ambisonic_files, batch_noise_files, batch_metadata, output_path, duration, sr, i // batch_size))

        # Wait for all futures to complete
        for future in futures:
            future.result()

    print("All batches processed and saved in parallel.")



# DataLoaders

In [41]:
class SoundLocalizationDataset(Dataset):
    def __init__(self, labels_df, base_path):
        """
        labels_df: DataFrame containing labels and corresponding batch_ids.
        base_path: Path where batch data files are stored.
        """
        self.labels_df = labels_df
        self.base_path = base_path

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        # Fetch the batch id for this index
        batch_id = self.labels_df.iloc[idx]['batch_id']

        # Load the corresponding batch data
        data_path = f'{self.base_path}/processed_batch_{batch_id}.pt'
        batch_data = torch.load(data_path)

        # Calculate the index within the batch
        local_idx = idx % len(batch_data)

        # Get the sample and its label
        sample = batch_data[local_idx]
        label = self.labels_df.iloc[idx][['azimuth', 'elevation']].values.astype('float32')

        return sample, label

In [42]:
def split_data(labels_df, test_size=0.2):
    train_df = labels_df[labels_df['split'] == 'train']
    test_df = labels_df[labels_df['split'] == 'test']
    train_df, valid_df = train_test_split(train_df, test_size=test_size, random_state=42)
    return train_df, valid_df, test_df


In [ ]:
filtered_metadata, ambisonic_files, noise_files = filter_data()
duration = filtered_metadata["audio_info/duration"].mean() + filtered_metadata["audio_info/duration"].std()
process_batches(filtered_metadata, ambisonic_files, noise_files, duration)


# Example usage
data_path = "/content/drive/My Drive/Colab Notebooks/Masters Project/spatial_librispeech_sample/preprocessed_samples"
labels_path = "/content/drive/My Drive/Colab Notebooks/Masters Project/spatial_librispeech_sample/preprocessed_samples"
batch_size = 32

labels_df = pd.read_csv(labels_path)
labels_df['batch_id'] = labels_df.index // batch_size  # Assuming labels are ordered as per batch

train_df, valid_df, test_df = split_data(labels_df)

train_dataset = SoundLocalizationDataset(data_path, train_df)
valid_dataset = SoundLocalizationDataset(data_path, valid_df)
test_dataset = SoundLocalizationDataset(data_path, test_df)

# DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
first_batch_data, first_batch_labels = next(iter(train_loader))
print("First batch data:", first_batch_data)
print("First batch labels:", first_batch_labels)